In [1]:
import open3d as o3d
import numpy as np

filepath = './Demo/cloudscloudscloudsclouds/brandon_fast/brandon_5_8_fast.ply'
pcd = o3d.io.read_point_cloud(filepath)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#pcd.paint_uniform_color([0.4,0.9,0.2])
#o3d.visualization.draw_geometries([pcd])

In [3]:
print(pcd)
print(np.asarray(pcd.points))

PointCloud with 1001813 points.
[[ 0.42494208 -0.400855   -0.2942968 ]
 [-0.03724247  0.14489147 -0.28252417]
 [ 0.13546094  0.28197092 -0.29478168]
 ...
 [ 0.45157975 -0.04539937 -0.28279302]
 [-0.17483065  0.93568194 -0.24286169]
 [-0.830616   -0.34507227 -0.28207886]]


In [4]:
#Floor remover: find plane and cut everything below it
plane_model, inliers = pcd.segment_plane(distance_threshold=0.003,
                                         ransac_n=5,
                                         num_iterations=2000)

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
#box = inlier_cloud.get_axis_aligned_bounding_box() #Creates bounding box for point cloud
#bounds = np.asarray(box.get_box_points()) #Get points for bounding box
#bounds = bounds[bounds[:,2].argsort()]
#for i in range(4):
#    bounds[i][2] = pcd.get_min_bound()[2]
#cpcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bounds)), invert=True)

PointCloud with 236519 points.

In [5]:
box = pcd.get_axis_aligned_bounding_box() #Creates bounding box for point cloud
inlierbox = inlier_cloud.get_axis_aligned_bounding_box()
bounds = np.asarray(box.get_box_points()) #Get points for bounding box
inlierbounds = np.asarray(inlierbox.get_box_points())
bounds = bounds[bounds[:,2].argsort()]
inlierbounds = inlierbounds[inlierbounds[:,2].argsort()]

for i in range(4):
    bounds[(i+4)][2]= inlierbounds[(i+4)][2]
cpcd = pcd.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(bounds)), invert=True)

In [6]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=15,
                                                    std_ratio=2.0)
print(cpcd)

PointCloud with 194826 points.


In [7]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=1.7)
print(cpcd)

PointCloud with 181771 points.


In [8]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=25,
                                                    std_ratio=1.6)
print(cpcd)

PointCloud with 178246 points.


In [9]:
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=100,
                                                    std_ratio=1.3)
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=85,
                                                    std_ratio=1.6)
cpcd, ind = cpcd.remove_statistical_outlier(nb_neighbors=85,
                                                    std_ratio=1.4)

In [10]:
cloud_low_bound = cpcd.get_min_bound()[2]
o3d.visualization.draw_geometries([cpcd])

In [11]:
cpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
#o3d.visualization.draw_geometries([cpcd],
  #                                point_show_normal=True)

In [19]:
d = 7
pmesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(cpcd, depth=d, linear_fit=True)
#WIP
#meshbounds = np.asarray(pmesh.get_axis_aligned_bounding_box().get_box_points())
#meshbounds = meshbounds[meshbounds[:,2].argsort()]
#for i in range(4):
#    meshbounds[(i+4)][2] = cloud_low_bound
#cpmesh = pmesh.crop(o3d.geometry.AxisAlignedBoundingBox.create_from_points(o3d.utility.Vector3dVector(meshbounds)), invert=True)

In [20]:
print(pmesh)
#print(cpmesh)

TriangleMesh with 6657 points and 13186 triangles.


In [21]:
pmesh.paint_uniform_color([0.1,0.7,0.5])
o3d.visualization.draw_geometries([pmesh], mesh_show_wireframe=True)
#cpmesh.paint_uniform_color([0.1,0.7,0.5])
#o3d.visualization.draw_geometries([cpmesh], mesh_show_wireframe=True)